In [1]:
import numpy as np
import pandas as pd

In [102]:
# import data
df = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/analysis.p')
subdf = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/subanalysis.p')
# NOTE THAT 'IN' MEAN IN TO THE ALT DISTRIBUTION, WHCIH IS NOT WHAT MATHEMATICA DOES

In [3]:
def getcount(decision, direction):
    directiondir = {'in':'-1', 'out':'1', 'unk':'0', 'fail':'2'}
    query = '%s == %s' %(decision, directiondir[direction])
    num = len(np.asarray(df.query(query)[decision], dtype=int))
    return num

In [4]:
# plot some stuff
expin = getcount('dexp', 'in')
expout = getcount('dexp', 'out')
expunk = getcount('dexp', 'unk')
lnin = getcount('dln', 'in')
lnout = getcount('dln', 'out')
lnunk = getcount('dln', 'unk')
strexpin = getcount('dstrexp', 'in')
strexpout = getcount('dstrexp', 'out')
strexpunk = getcount('dstrexp', 'unk')
plwcin = getcount('dplwc', 'in')
plwcout = getcount('dplwc', 'out')
plwcunk = getcount('dplwc', 'unk')
# order = "LN", "Exp", "Str Exp", "PLwC"
incounts = [lnin, expin, strexpin, plwcin]
outcounts = [lnout, expout, strexpout, plwcout]
unkcounts = [lnunk, expunk, strexpunk, plwcunk]
print incounts
print outcounts
print unkcounts

[415, 317, 388, 489]
[136, 1069, 852, 0]
[1239, 301, 500, 1333]


In [52]:
doms = np.unique(df.Domain)
# number of graphs and mean alpha value in each domain
counts = np.zeros_like(doms)
means = np.zeros_like(doms)
counts_plaus = np.zeros_like(doms)
means_plaus = np.zeros_like(doms)
counts_plaustail = np.zeros_like(doms)
means_plaustail = np.zeros_like(doms)
for i,dom in enumerate(doms):
    query = "Domain == '%s'" %dom
    subdf = df.query(query)
    counts[i] = len(subdf)
    means[i] = format(np.mean(subdf.alpha), '.2f')
    subdf_plaus = subdf.query("ppl>0.1")
    counts_plaus[i] = len(subdf_plaus)
    means_plaus[i] = format(np.mean(subdf_plaus.alpha), '.2f')
    subdf_plaustail = subdf_plaus.query("ntail>=50")
    counts_plaustail[i] = len(subdf_plaustail)
    means_plaustail[i] = format(np.mean(subdf_plaustail.alpha), '.2f')


In [72]:
print doms
print counts_plaustail
print means_plaustail

['Biological' 'Economic' 'Informational' 'Social' 'Technological'
 'Transportation']
[53 0 6 106 799 1]
['2.69' 'nan' '2.38' '4.74' '2.23' '4.82']


In [114]:
def test_strong(rows):
    SA = False # strong alone
    S1 = False # strong 1
    S2 = False # strong 2
    n = len(rows)
    strong = 0
    strong_alone = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1 and row.ntail >= 50 and row.alpha < 3 and row.alpha > 2:
            strong_alone += 1
            if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1  and row.dplwc >-1:
                strong += 1
    if strong_alone >= n/2:
        SA = True
    if strong >= n/2:
        S2 = True
    if strong >= 9*n/10:
        S1 = True
    return (S1, S2, SA)
            
def test_weak(rows):
    W = False 
    West = False 
    n = len(rows)
    weak = 0
    weakest = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1:
            weakest += 1
            if row.ntail>=50:
                weak += 1
    if weak >= n/2:
        W = True
    if weakest >= n/2:
        West = True
    return (W, West)
            
          

In [138]:
# test diff hypotheses
unique_datasets = np.unique(df.fp_gml)
hyps = pd.DataFrame(columns = ['Strong1', "Strong2", "Strong_alone", "Weak", "Weakest", "Domain", "Subdomain", "mean_alpha"], index=unique_datasets )
for i, dataset in enumerate(unique_datasets):
    query = "fp_gml == '%s'" %dataset
    rows = df.query(query)
    [S1, S2, SA] = test_strong(rows)
    hyps.loc[dataset]['Strong1'] = S1
    hyps.loc[dataset]['Strong2'] = S2
    hyps.loc[dataset]['Strong_alone'] = SA
    [weak,weakest] = test_weak(rows)
    hyps.loc[dataset]['Weak'] = weak
    hyps.loc[dataset]['Weakest'] = weakest
    hyps.loc[dataset]['Domain'] = rows.Domain[0]
    hyps.loc[dataset]['Subdomain'] = rows.Subdomain[0]

In [140]:
dataset = unique_datasets[0]
query = "fp_gml == '%s'" %dataset
rows = df.query(query)
[S1, S2, SA] = test_strong(rows)
hyps.loc[dataset]['Strong1'] = S1
hyps.loc[dataset]['Strong2'] = S2
hyps.loc[dataset]['Strong_alone'] = SA
[weak,weakest] = test_weak(rows)
hyps.loc[dataset]['Weak'] = weak
hyps.loc[dataset]['Weakest'] = weakest
hyps.loc[dataset]['Domain'] = rows.Domain[0]
hyps.loc[dataset]['Subdomain'] = rows.Subdomain[0]
hyps.loc[dataset]['mean_alpha'] = np.mean(rows.alpha)
hyps.loc[dataset]
rows

,Domain,Subdomain,num_edges,Graph_order,Weighted,Directed,Bipartite,Multigraph,Multiplex,Component,...,n,alpha,xmin,ntail,Lpl,ppl,dexp,dln,dstrexp,dplwc
Cat_cerebral_hemisphere_cortex_and_thalamus_Biological_Connectome_n2.gml_weighted1_directedindistribution.txt,Biological,Connectome,244,2,w1,in,0,0,0,entire,...,76,6.5,6,13,-14.4819,0.478,-1,0,-1,0
Cat_cerebral_hemisphere_cortex_and_thalamus_Biological_Connectome_n2.gml_weighted1_directedoutdistribution.txt,Biological,Connectome,244,2,w1,out,0,0,0,entire,...,76,5.35,5,20,-26.3621,0.837,0,0,0,0
Cat_cerebral_hemisphere_cortex_and_thalamus_Biological_Connectome_n2.gml_weighted1_directedtotaldistribution.txt,Biological,Connectome,244,2,w1,total,0,0,0,entire,...,76,1.99,3,59,-174.264,0,-1,-1,-1,-1


In [133]:
n = float(len(df))
Strong1 = np.sum(hyps.Strong1)
Strong2 = np.sum(hyps.Strong2)
Strong_alone = np.sum(hyps.Strong_alone)
Weak = np.sum(hyps.Weak)
Weakest = np.sum(hyps.Weakest)
print "Strong1 = %s" %(Strong1/n)
print "Strong2 = %s" %(Strong2/n)
print "Strong_alone = %s" %(Strong_alone/n)
print "Weak = %s" %(Weak/n)
print "Weakest = %s" %(Weakest/n)
print "Number of Datsets = %s" %len(unique_datasets)

Strong1 = 0.229265967588
Strong2 = 0.261201143947
Strong_alone = 0.264061010486
Weak = 0.28360343184
Weakest = 0.322449952336
Number of Datsets = 1823


In [137]:
query = "fp_gml == '%s'" %'/Volumes/Durodon/gmls/Transportation/Airport/n4/Openflights_airport_network_2010_Openflights_airport_network_2010_Transportation_Airport_n4.gml'
rows = df.query(query)
np.mean(rows.alpha)

1.7400000000000009

In [160]:
set(hyps.Domain)

{'Biological',
 'Economic',
 'Informational',
 'Social',
 'Technological',
 'Transportation'}

In [ ]:
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
ax = hyps[['Strong1','Strong2']].plot(kind='bar')
plt.show()